Import di base

In [1]:
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from IPython.display import clear_output

import numpy as np
import random
import socket
import time

import struct



In [5]:
# definizioni variabili globali ==================================================================
HOST = "127.0.0.1"
PORT = 7001
PACKET_SIZE = 1024

PacketCounter = 0
PacketCounter_lastCheck = 0

# definizioni funzioni =======================================================

#creo socket e lo faccio partire
def CreaSocket():
    # creo socket server per UE5
    MySocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    #Bindo il socket 
    MySocket.bind((HOST,PORT))

    #accendo il socket in ascolto
    MySocket.listen()

    return MySocket


#ciclo di attesa connesione
def AttesaConnesione(MySocket):
    while True:
        RemoteConnection , RemoteAddress = MySocket.accept()
        MySocket.close()
        break
    return RemoteConnection,RemoteAddress

#funzione per sampolare il robot di unreal
def MandaRicevi_Robot(MySocket, R_Start=0.0, R_Arm_1=0.0, R_Arm_2=0.0, R_Arm_3=0.0, R_Arm_4=0.0, R_End=0.0):
    Messaggio =    (struct.pack("f",R_Start)+
                    struct.pack("f",R_Arm_1)+
                    struct.pack("f",R_Arm_2)+
                    struct.pack("f",R_Arm_3)+
                    struct.pack("f",R_Arm_4)+
                    struct.pack("f",R_End)
                    )
    MySocket.sendall(Messaggio)
    ReceivedData = MySocket.recv(PACKET_SIZE)

    return struct.unpack("f",ReceivedData)[0]

# classe environment di gioco =============================================

class RobotEnv(Env):
    def __init__(self):
        # azioni che possiamo fare
        self.action_space = Discrete(3)
        # Array distanze dal target
        self.observation_space = Box(low=-10000,high=10000,shape=(2,))
        # distanza di partenza
        self.state = [0.0,0.0]

        self.angoloAttuale_1 = random.uniform(-120,120)
        self.angoloAttuale_2 = random.uniform(0,90)

        # lunghezza esercizio
        self.Exercise_length = 800

    def step(self, action,verbosita=False):
        global UE5_Socket
        
        # applichiamo azione
        
        if action == 0:
            self.angoloAttuale_1 += 2
        elif action == 1:
            self.angoloAttuale_1 -= 2
        elif action == 2:
            pass
        '''
        if action == 3:
            self.angoloAttuale_2 += 2
        elif action == 4:
            self.angoloAttuale_2 -= 2
        elif action == 5:
            pass
        '''
        #self.state[2] = self.angoloAttuale_2
        self.state[1] = self.angoloAttuale_1
        self.state[0] = MandaRicevi_Robot(UE5_Socket, -90.0, self.angoloAttuale_1) #, self.angoloAttuale_2)

        # riduzione tempo 
        self.Exercise_length -= 1

        #calcolo reward
        if self.state[0] <= 40:
            reward = 1
        else:
            reward = -3 * (self.state[0] / 100)

        done = False

        #verfica se tempo finito
        if self.Exercise_length <= 0:
            done = True
        else:
            done = False
               
        # segnaposto per info
        info = {}
        
        # ritorniamo informazioin sullo step
        return self.state, reward, done, info

    def render(self,mode=0):
        print("Tempo: "+str(self.Exercise_length) + "-- state: " + str(self.state))

    def reset(self):
        #resettiamo temperatura
        self.angoloAttuale_1 = random.uniform(-120,120)
        self.angoloAttuale_2 = random.uniform(0,90)
        self.state = [0.0,0.0]
        # resettiamo tempo
        self.Exercise_length = 800
        return self.state

# codice ==================================================================

#creo e ascolto
Server_Socket = CreaSocket()

#attendo e connetto con UE5 
UE5_Socket , UE5_Address = AttesaConnesione(Server_Socket)

#confermo connesione aperta
print(f"Aperto socket con UE5: {UE5_Address}")


SimpleRobotEnv = RobotEnv()


Aperto socket con UE5: ('127.0.0.1', 50310)


In [7]:
# giro di prova dell'exercise
episodes = 1
for episode in range(1, episodes+1):
    state = SimpleRobotEnv.reset()
    done = False
    score = 0 
    
    while not done:
        SimpleRobotEnv.render(mode=0)
        action = SimpleRobotEnv.action_space.sample()
        n_state, reward, done, info = SimpleRobotEnv.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Tempo: 800-- state: [0.0, 0.0]
Tempo: 799-- state: [205.78248596191406, -68.94611112045637]
Tempo: 798-- state: [203.23684692382812, -66.94611112045637]
Tempo: 797-- state: [200.62966918945312, -64.94611112045637]
Tempo: 796-- state: [197.96176147460938, -62.946111120456365]
Tempo: 795-- state: [197.96176147460938, -62.946111120456365]
Tempo: 794-- state: [195.23394775390625, -60.946111120456365]
Tempo: 793-- state: [197.96176147460938, -62.946111120456365]
Tempo: 792-- state: [200.62966918945312, -64.94611112045637]
Tempo: 791-- state: [203.23684692382812, -66.94611112045637]
Tempo: 790-- state: [200.62966918945312, -64.94611112045637]
Tempo: 789-- state: [203.23684692382812, -66.94611112045637]
Tempo: 788-- state: [205.78248596191406, -68.94611112045637]
Tempo: 787-- state: [203.23684692382812, -66.94611112045637]
Tempo: 786-- state: [203.23684692382812, -66.94611112045637]
Tempo: 785-- state: [203.23684692382812, -66.94611112045637]
Tempo: 784-- state: [200.62966918945312, -64.94611

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*4)])
  except RuntimeError as e:
    print(e)

states = SimpleRobotEnv.observation_space.shape
actions = SimpleRobotEnv.action_space.n

def build_model(states,actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + SimpleRobotEnv.observation_space.shape))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(actions,activation="linear"))
    return model

In [9]:
del model # da usare quando da errore sotto

In [10]:
model = build_model(states,actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 195       
Total params: 8,707
Trainable params: 8,707
Non-trainable params: 0
_________________________________________________________________


In [11]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy
from rl.memory import SequentialMemory

def build_agent(model, actions):
    policy = BoltzmannGumbelQPolicy()
    memory = SequentialMemory(limit=50000,window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

In [12]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3),metrics=["mae"])
dqn.fit(SimpleRobotEnv, nb_steps=500000,visualize=False,verbose=1)

Training for 500000 steps ...
Interval 1 (0 steps performed)
   10/10000 [..............................] - ETA: 6:15 - reward: -1.5636 

c:\Users\xmari\.conda\envs\aaaa\lib\site-packages\rl\memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 1899/10000 [====>.........................] - ETA: 2:14 - reward: -0.1280done, took 31.637 seconds


In [13]:
scores = dqn.test(SimpleRobotEnv, nb_episodes=10,visualize=False)
print(np.mean(scores.history["episode_reward"]))

Testing for 10 episodes ...
Episode 1: reward: 800.000, steps: 800
Episode 2: reward: 695.470, steps: 800
Episode 3: reward: 507.072, steps: 800
Episode 4: reward: 760.134, steps: 800
Episode 5: reward: 649.863, steps: 800
Episode 6: reward: 761.502, steps: 800
Episode 7: reward: 711.088, steps: 800
Episode 8: reward: 701.224, steps: 800
Episode 9: reward: 752.858, steps: 800
Episode 10: reward: 740.024, steps: 800
707.9235928726196


Chiusure varie

In [ ]:
model.save("modello.h5")

new_model = tf.keras.models.load_model("modello.h5")

In [ ]:
# spengo socket prima di uscire
UE5_Socket.shutdown(socket.SHUT_RDWR)
UE5_Socket.close()

Codice vario utile:

In [ ]:
'''
DistanzaAttuale = MandaRicevi_Robot(
                            UE5_Socket,
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0)
                            )
'''

'\nDistanzaAttuale = MandaRicevi_Robot(\n                            UE5_Socket,\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0)\n                            )\n'